## Training

### Import Libraries and functions from utils.py

In [20]:
import pandas as pd
import scipy
import sklearn
from sklearn import *
import numpy as np
import os
from utils import *
import pickle
from sklearn.linear_model import LogisticRegression
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Callable, Tuple

In [11]:
path_data =  os.path.expanduser('~') 

# use this to train and VALIDATE your solution
train_df = pd.read_csv("./data/quora_train_data.csv")

# use this to provide the expected generalization results
test_df = pd.read_csv("./data/quora_test_data.csv")

In [12]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,346692,38482,10706,Why do I get easily bored with everything?,Why do I get bored with things so quickly and ...,1
1,327668,454117,345117,How do I study for Honeywell company recruitment?,How do I study for Honeywell company recruitme...,1
2,272993,391373,391374,Which search engine algorithm is Quora using?,Why is Quora not using reliable search engine?,0
3,54070,82673,95496,How can I smartly cut myself?,Can someone who thinks about suicide for 7 yea...,0
4,46450,38384,72436,How do I see who is viewing my Instagram videos?,Can one tell who viewed my Instagram videos?,1


In [13]:
#TODO: don't we load the test set to test_df?
A_df, te_df = sklearn.model_selection.train_test_split(train_df, test_size=0.05,random_state=123)

tr_df, va_df = sklearn.model_selection.train_test_split(A_df, test_size=0.05,random_state=123)
print('tr_df.shape=',tr_df.shape)
print('va_df.shape=',va_df.shape)
print('te_df.shape=',te_df.shape)

tr_df.shape= (291897, 6)
va_df.shape= (15363, 6)
te_df.shape= (16172, 6)


<b>cast_list_as_strings</b> casts each element in the input list to a string.

In [14]:
q1_train =  cast_list_as_strings(list(train_df["question1"]))
q2_train =  cast_list_as_strings(list(train_df["question2"]))
q1_test  =  cast_list_as_strings(list(test_df["question1"]))
q2_test  =  cast_list_as_strings(list(test_df["question2"]))

In [15]:
q1_train[0], q2_train[0]

('Why do I get easily bored with everything?',
 'Why do I get bored with things so quickly and easily?')

Use all the questions in train and test partitions to create a single <b>list all_questions to fit the TfidfVectorizer.</b>

In [16]:

# Preprocess the training data
preprocessed_train_questions_1 = [preprocess_text(question) for question in q1_train]
preprocessed_train_questions_2 = [preprocess_text(question) for question in q2_train]

# Create TF-IDF features for the training data
vectorizer = TfidfVectorizer()
questions_tfidf = vectorizer.fit_transform(preprocessed_train_questions_1 + preprocessed_train_questions_2)

# Train the logistic regression model
model = LogisticRegression()
X_train = np.abs(questions_tfidf[:len(q1_train)] - questions_tfidf[len(q1_train):])
y_train = train_df["is_duplicate"].values
model.fit(X_train, y_train)

/home/zozoka/anaconda3/envs/quora_challange_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [17]:
save_model_with_overwrite(model, 'model_artifacts/logistic_adv_model.joblib')


Model saved to model_artifacts/logistic_adv_model.joblib


In [26]:
def same_question_word_custom_similarity(questions_1: List[str], questions_2: List[str]) -> List[float]:
    question_words = {"who", "what", "when", "where", "why", "how"}
    similarity_scores = []
    return similarity_scores